In [3]:
import pathlib
import json
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import defaultdict
from text_processing.TextProcessor import TextProcessor
from Tfidf.Tf_idf_Service import TfidfService
from Ranking.ranking_service import RankingService

# Instantiate services
tfidf_service = TfidfService()
textprocessor = TextProcessor()
rankingservice = RankingService()

# Load the saved vectorizer, TF-IDF matrix, and corpus
cwd = pathlib.Path().cwd()
clinical_tfidf_matrix_path = cwd / 'files' / 'clinicaltrials' / 'tfidf_matrix.pkl'
lifestyle_tfidf_matrix_path = cwd / 'files' / 'lifestyle' / 'tfidf_matrix.pkl'
clinical_corpus_key_path = cwd / 'files' / 'clinicaltrials' / 'corpuskey.json'
lifestyle_corpus_key_path = cwd / 'files' / 'lifestyle' / 'corpuskey.json'
lifestyle_folder = cwd / "files" / "lifestyle/"
clinical_folder = cwd / "files" / "clinicaltrials/"
tfidf_service.preload(lifestyle_folder, clinical_folder)

with open(lifestyle_corpus_key_path, 'r') as f:
    lifestyle_corpus_keys = json.load(f)

def docsIdsSearch(query_text, corpus_keys, dataset):
    processed_query = textprocessor.process_text(query_text)
    query_vector = tfidf_service.vectorize_query_evaluation(processed_query, dataset)
    similarity_scores = tfidf_service.calculate_similarity(query_vector, dataset)
    sorted_ranks = rankingservice.rank_and_sort(similarity_scores)
    return [corpus_keys[doc_idx] for doc_idx, _ in sorted_ranks]


Preloading successful


In [7]:
def load_qrel_lifestyle(jsonl_file_path):
    qrel_dict = defaultdict(list)
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            entry = json.loads(line.strip())
            query_id = entry["qid"]
            answer_pids = entry["answer_pids"]
            qrel_dict[query_id] = answer_pids
    return qrel_dict

# Paths to QREL files
lifestyle_qrel_path = cwd / "files" / 'lifestyle' / "qas.search.jsonl"

# Load QREL data
lifestyle_qrel_dict = load_qrel_lifestyle(lifestyle_qrel_path)


In [8]:
# Add function to retrieve lifestyle documents
retrieved_docs_lifestyle = {}
print("Retrieving documents for each lifestyle query...")
dataset = "lifestyle"
with open(lifestyle_qrel_path, 'r', encoding='utf-8') as file:
    for line in file:
        entry = json.loads(line.strip())
        query_id = entry["qid"]
        query_text = entry["query"]
        ret_docs = docsIdsSearch(query_text, lifestyle_corpus_keys, dataset)
        retrieved_docs_lifestyle[query_id] = ret_docs
print("Lifestyle document retrieval complete.")


Retrieving documents for each lifestyle query...
Lifestyle document retrieval complete.


In [9]:
class EvaluationMetrics:
    def __init__(self, true_data, predictions):
        self.true_data = {str(k): list(map(str, v)) for k, v in true_data.items()}
        self.predictions = {str(k): list(map(str, v)) for k, v in predictions.items()}

    def calculate_recall(self, true_pids, pred_indices):
        true_set = set(true_pids)
        pred_set = set(pred_indices)
        if len(true_set) == 0:
            return 0
        return len(true_set & pred_set) / len(true_set)

    def calculate_precision_at_k(self, true_pids, pred_indices, k):
        true_set = set(true_pids)
        pred_set = set(pred_indices[:k])
        if len(pred_set) == 0:
            return 0
        return len(true_set & pred_set) / k

    def average_precision(self, true_pids, pred_indices):
        relevant = 0
        sum_precisions = 0
        for i, pred in enumerate(pred_indices):
            if pred in true_pids:
                relevant += 1
                sum_precisions += relevant / (i + 1)
        if relevant == 0:
            return 0
        return sum_precisions / len(true_pids)

    def mean_reciprocal_rank(self, true_pids, pred_indices):
        for rank, pid in enumerate(pred_indices, start=1):
            if pid in true_pids:
                return 1 / rank
        return 0

    def calculate_metrics(self):
        recalls = []
        precisions_k = []
        aps = []
        mrrs = []

        for query_id, true_ids in self.true_data.items():
            pred_ids = self.predictions.get(query_id, [])
            recalls.append(self.calculate_recall(true_ids, pred_ids))
            precisions_k.append(self.calculate_precision_at_k(true_ids, pred_ids, 10))
            aps.append(self.average_precision(true_ids, pred_ids))
            mrrs.append(self.mean_reciprocal_rank(true_ids, pred_ids))

        mean_recall = sum(recalls) / len(recalls)
        mean_precision_at_k = sum(precisions_k) / len(precisions_k)
        mean_ap = sum(aps) / len(aps)
        mean_mrr = sum(mrrs) / len(mrrs)

        print(f"Mean Recall: {mean_recall:.4f}")
        print(f"Precision@10: {mean_precision_at_k:.4f}")
        print(f"Mean Average Precision: {mean_ap:.4f}")
        print(f"Mean Reciprocal Rank: {mean_mrr:.4f}")

        return mean_recall, mean_precision_at_k, mean_ap, mean_mrr

print("Evaluating lifestyle metrics...")
evaluation_lifestyle = EvaluationMetrics(lifestyle_qrel_dict, retrieved_docs_lifestyle)
mean_recall, mean_precision_at_k, mean_ap, mean_mrr = evaluation_lifestyle.calculate_metrics()
print("retrieved_docs_lifestyle saved as JSON.")


Evaluating lifestyle metrics...
Mean Recall: 1.0000
Precision@10: 0.0803
Mean Average Precision: 0.1837
Mean Reciprocal Rank: 0.3095
retrieved_docs_lifestyle saved as JSON.
